# Classification with RIPPER
In this notebook we aim to build a classifier using RIPPER. RIPPER is one of the most popular rule based classifier.

In [8]:
import pandas as pd
from os import path
import numpy as np
from preprocessing import get_train_test_data

X_train, y_train, X_test, y_test, columns_to_keep = get_train_test_data()

To begin, we create a target variable named `top_20` based on the column `position`.

Now we need to discretize the data to make it suitable for RIPPER. We will use the `discretize_data` function to discretize the data.

Finally, we will train the RIPPER classifier using the discretized data.

In [9]:
#function to discretize the variables
#input: the dataset and the list of variables' names to discretize
def discretize_data(dataset, variables):
    for variable in variables:
        #get the unique variable's values
        var = sorted(dataset[variable].unique())

        #generate a mapping from the variable's values to the number representation
        mapping = dict(zip(var, range(0, len(var) + 1)))

        #add a new colum with the number representation of the variable
        dataset[variable+'_num'] = dataset[variable].map(mapping).astype(int)
    return dataset

In [13]:
X_train = discretize_data(X_train, [col for col in columns_to_keep if col != "top_20"])

X_test = discretize_data(X_test, [col for col in columns_to_keep if col != "top_20"])

X_train.head()

,bmi,career_points,career_duration(days),debut_year,difficulty_score,competitive_age,is_tarmac,points,climbing_efficiency,startlist_quality,bmi_num,career_points_num,career_duration(days)_num,debut_year_num,difficulty_score_num,competitive_age_num,is_tarmac_num,points_num,climbing_efficiency_num,startlist_quality_num
0,23.765432,68034.221635,6233.0,1977.0,0.635375,22,True,100.0,0.006796,1241,501,2810,2476,7,83,3,1,5,65,296
1,20.897959,29429.221635,5212.0,1974.0,0.635375,27,True,100.0,0.006796,1241,248,2331,2329,4,83,8,1,5,65,296
2,22.790329,15880.000000,2972.0,1977.0,0.635375,24,True,100.0,0.006796,1241,429,1703,1571,7,83,5,1,5,65,296
3,21.469150,6600.000000,3606.0,1970.0,0.635375,30,True,100.0,0.006796,1241,303,908,1826,0,83,11,1,5,65,296
4,21.295295,17245.000000,2192.0,1977.0,0.635375,27,True,100.0,0.006796,1241,287,1779,1168,7,83,8,1,5,65,296


Now we search for the best hyperparemeters for the RIPPER classifier using the `GridSearchCV` function.

Finally, we evaluate the model using the `classification_report` function.

As we can see from the classification report, the RIPPER classifier has an accuracy of 0.85. Unfortunately, while the precision and recall are high for the negative class, they are low for the positive class. This is likely due to the class imbalance in the dataset. We try to address this issue by using the `class_weight` parameter in the RIPPER classifier.

In [18]:
import wittgenstein as lw
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score



model = lw.RIPPER()

param_grid = {
    'k': [1, 2],
    'max_rules': [10, 20],
    'prune_size': [0.33],
    'class_weights': [{0:1, 1:5}, {0:10}, {0: 1, 1: 25}]
}

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,
    verbose=1
)

grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)


report = classification_report(y_test, y_pred)
best_params = grid_search.best_params_
accuracy = accuracy_score(y_test, y_pred)


Fitting 3 folds for each of 12 candidates, totalling 36 fits


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
print(accuracy)

0.8511737782371179


In [20]:
print(best_params)

{'class_weights': {0: 1, 1: 25}, 'k': 1, 'max_rules': 20, 'prune_size': 0.33}


In [21]:
print(report)

              precision    recall  f1-score   support

           0       0.85      1.00      0.92      5765
           1       0.00      0.00      0.00      1008

    accuracy                           0.85      6773
   macro avg       0.43      0.50      0.46      6773
weighted avg       0.72      0.85      0.78      6773



Now we print the rules generated by the model. The rules provide insight into the decision making process of the model.

In [22]:
# Visualizzazione strutturata
for rule in best_model.ruleset_:
    print(rule)


[debut_year=<1982.0^points=<50.0]
[startlist_quality=<705.0^debut_year=<1982.0^climbing_efficiency=0.012-0.013]
[startlist_quality=<705.0^debut_year=<1982.0^is_tarmac=False^difficulty_score=1.5-1.71^career_points=14794.22-19135.0^bmi=22.2-22.79]
[startlist_quality=<705.0^debut_year=<1982.0^career_duration(days)=3689.0-4108.0^competitive_age=<24.0^bmi=21.78-22.2]
[startlist_quality=<705.0^debut_year=<1982.0^is_tarmac=False^difficulty_score=1.5-1.71^career_duration(days)=3280.0-3689.0]
[startlist_quality=<705.0^debut_year=1982.0-1989.0^points=<50.0]


Again, with the new class weights, we evaluate the model using the `classification_report` function. The accuracy of the model is still 0.85, but the scores are the same as before. This suggests that the the model is to simple to capture the complexity of the data.